# 合并ksrpc+frpc两阻塞代码于单Notebook
1. 支持子程序崩溃重新拉起
2. 支持空闲超时停止服务
3. 支持日志超长清屏

In [ ]:
from ksrpc.run_notebook import main

commands = {
    "ksrpc": ["python", "-u", "-m", "ksrpc.run_app"],
    "frpc": ["./frpc", "-c", "./frpc.toml"],
}
main(commands, idle_timeout=300, clear_count=35)